In [40]:
from langchain.chat_models.openai import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler

llm = ChatOpenAI(
    temperature=0.1,
    model="gpt-3.5-turbo-0125",
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

In [41]:
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.vectorstores import FAISS


def embed_file():
    cache_dir = LocalFileStore("./.cache/")
    loader = UnstructuredFileLoader("./files/document.txt")
    splitter = CharacterTextSplitter.from_tiktoken_encoder(
        separator="\n",
        chunk_size=600,
        chunk_overlap=50,
    )
    docs = loader.load_and_split(text_splitter=splitter)
    embedder = OpenAIEmbeddings()
    cache_embedder = CacheBackedEmbeddings.from_bytes_store(embedder, cache_dir)
    vectorStore = FAISS.from_documents(docs, cache_embedder)
    retriever = vectorStore.as_retriever()

    return retriever

In [42]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    return_messages=True,
    memory_key="history"
)

def save_memory(input, output):
    memory.save_context({"input": input}, {"output" :  output})

def load_memory(_):
    return memory.load_memory_variables({})["history"]

In [43]:
def format_doc(document):
    return "\n\n".join(doc.page_content for doc in document)

In [44]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a helpful assistant. Answer questions using only the following context. and You remember conversations with human.
            DON'T make it up.
            --------
            {context}
            """,
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

retriever = embed_file()

chain = (
    {
        "context": retriever | RunnableLambda(format_doc),
        "history": RunnableLambda(load_memory),
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
)


def invoke_chain(question):
    result = chain.invoke(question).content
    save_memory(question, result)

libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


In [45]:
invoke_chain("Is Aaronson guilty?")

According to the context, Winston believes that Jones, Aaronson, and Rutherford are guilty of the crimes they are charged with, and he has convinced himself that he had never seen the photograph that disproved their guilt, which he had invented. However, this belief is a product of the Party's manipulation and the process of crimestop, where he trains himself to accept the Party's version of reality without question.

In [46]:
invoke_chain("What message did he write in the table?")

Winston wrote the following messages on the slate: 

1. FREEDOM IS SLAVERY
2. TWO AND TWO MAKE FIVE
3. GOD IS POWER

In [47]:
invoke_chain("Who is Julia?")

Julia is a character who is Winston's love interest. In the context, she represents a connection to love and rebellion against the oppressive regime of the Party. Winston experiences a deep emotional connection with her, and despite the dangers of their relationship, he feels a strong desire to be with her. Julia is also someone he believes needs his help, indicating a bond that goes beyond mere physical attraction.